In [93]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime

In [27]:
### root directory
root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
path_data = os.path.join(root, 'data')
path_oxcgrt = os.path.join(path_data, 'OxCGRT_timeseries_all.xlsx')

In [29]:
## open excel 
EXCEL = pd.ExcelFile(path_oxcgrt)

## get sheet names
EXCEL.sheet_names

['stringency_index_avg',
 'containment_health_index_avg',
 'government_response_index_avg',
 'economic_support_index',
 'c1m_school_closing',
 'c1m_flag',
 'c2m_workplace_closing',
 'c2m_flag',
 'c3m_cancel_public_events',
 'c3m_flag',
 'c4m_restrictions_on_gatherings',
 'c4m_flag',
 'c5m_close_public_transport',
 'c5m_flag',
 'c6m_stay_at_home_requirements',
 'c6m_flag',
 'c7m_movementrestrictions',
 'c7m_flag',
 'c8ev_internationaltravel',
 'e1_income_support',
 'e1_flag',
 'e2_debtrelief',
 'h1_public_information_campaigns',
 'h1_flag',
 'h2_testing_policy',
 'h3_contact_tracing',
 'h6m_facial_coverings',
 'h6m_flag',
 'h7_vaccination_policy',
 'h7_flag',
 'h8m_protection_of_elderly_ppl',
 'h8m_flag',
 'confirmed_cases',
 'confirmed_deaths']

In [94]:
## read sheet
selected_sheet = 'h2_testing_policy' # c4m_restrictions_on_gatherings

df = EXCEL.parse(selected_sheet)

In [95]:
df = (df
      .query('country_code == "USA" | country_code == "IND" | country_code == "IDN"') # only need USA, india and indonesia
      .query('jurisdiction == "NAT_TOTAL"') # only need national level
      .drop(['country_name', 'region_code', 'region_name', 'jurisdiction'], axis=1)
      .reset_index(drop=True) # tidy up
      
      .set_index(['country_code']) # melting
      .unstack() # ibid
      .reset_index() # ibid
      .rename(columns={'level_0': 'date', 0: 'level'}) # rename columns
      .sort_values(['country_code', 'date']) # sorting
      .reset_index(drop=True) # tidy up
      .assign(date = lambda x: x['date'].apply(lambda x: datetime.strptime(x, '%d%b%Y').strftime('%Y-%m-%d'))) # transform 01Apr2022 to 2022-04-01
      )

In [96]:
df.to_csv(os.path.join(path_data, f'oxcgrt_{selected_sheet}.csv'), index=False)